In [15]:
import pandas as pd
import numpy as np

In [7]:
closing_price_data = pd.read_pickle("./closing_price_5yr.pkl").dropna(axis = 1)

In [8]:
stock_names = closing_price_data.keys()

# Now is for the partitions of data

In [132]:
from collections import OrderedDict

def optimal_solution(stock_data_partition):
    logret = np.log(stock_data_partition).diff()
    mu = logret[1:].mean()
    sigma = logret.cov()
    s, _ = sigma.shape
    w = cvx.Variable(s)# the length of w is x
    risk = cvx.quad_form(w, sigma.as_matrix())
    prob = cvx.Problem(cvx.Minimize(risk), 
                   [cvx.sum_entries(w) == 1])
    prob.solve()
    
    return w.value
    

w=[]
def Optimization_At_Each_Period(stock_data,days_of_period):
    
    for current_period_num in range(int(np.floor( (len(stock_data)/days_of_period) ))):
        stock_data_partition = stock_data[current_period_num*days_of_period:(current_period_num+1)*days_of_period]
        w.append(optimal_solution(stock_data_partition))
    if len(stock_data)/days_of_period-int(len(stock_data)/days_of_period)==0:
         
        return np.column_stack(w)
    else:
        print ("Last Trading Period Not Finished")
        last_start=int(np.floor( (len(stock_data)/days_of_period) ))*days_of_period
        last_end=len(stock_data)-1
        w.append(optimal_solution(stock_data[last_start:last_end]))
        return np.column_stack(w)

re=[]
def return_over_time(stock_data,days_of_period):
    for current_period_num in range(int(np.floor( (len(stock_data)/days_of_period) ))):
        tmp=np.log(np.matrix(closing_price_data[(current_period_num+days_of_period):(current_period_num+1+days_of_period)])/np.matrix(closing_price_data[current_period_num:current_period_num+1]))
        re.append(tmp)
        
    if len(stock_data)/days_of_period-int(len(stock_data)/days_of_period)==0:
         
        return np.stack(re)
    else:
        print ("Last Trading Period Not Finished")
        last_start=int(np.floor( (len(stock_data)/days_of_period) ))*days_of_period
        last_end=len(stock_data)-1
        tmp=np.log(np.matrix(closing_price_data[last_end:last_end+1])/np.matrix(closing_price_data[last_start:last_start+1]))
        re.append(tmp)
        return np.stack(re)
    
def Realized_Return(stock_data,days_of_period):
    Mat_Optimization_At_Each_Period = Optimization_At_Each_Period(stock_data,days_of_period)
    Mat_return_over_time = return_over_time(stock_data,days_of_period)
    Total_time = Mat_Optimization_At_Each_Period.shape[1]
    average_return=0
    for ticker in range(Total_time):
        average_return =average_return+1/Total_time* np.matmul(Mat_return_over_time[ticker,:],
                                                              Mat_Optimization_At_Each_Period[:,ticker])
    return float(average_return)
    

In [89]:
int(np.floor( (len(closing_price_data)/21) ))*21

1239

In [90]:
closing_price_data.shape

(1259, 467)

In [112]:
ROT=return_over_time(closing_price_data,21)

Last Trading Period Not Finished


In [113]:
ROT.shape

(60, 467)

In [121]:
OMT= Optimization_At_Each_Period(closing_price_data,21)

Last Trading Period Not Finished


In [133]:
Realized_Return(closing_price_data,21)

Last Trading Period Not Finished
Last Trading Period Not Finished


0.004369884914259381

In [122]:
OMT.shape

(467, 60)

In [94]:
RR1.shape[1]

60

In [128]:
RR1[0,:]

matrix([[-9.53797244e-04, -3.83649229e-04,  8.92961527e-03,
          2.86074908e-03,  3.88661621e-03, -1.32410251e-03,
          4.32914470e-03,  2.27885964e-03,  1.23609991e-02,
         -3.04084652e-03,  1.25996131e-04, -1.47785254e-03,
          1.90988760e-03,  3.85890385e-04, -5.90042537e-04,
          7.44987420e-03,  9.27998649e-04,  1.09348819e-03,
         -9.68070851e-03, -2.79729471e-03, -7.91191332e-03,
         -3.66571280e-03,  2.35153390e-03,  4.41303030e-04,
         -2.70161982e-03, -8.51142985e-03,  6.29290085e-04,
         -1.09180453e-02, -6.88083427e-03,  2.41335448e-03,
          1.43105681e-03,  5.47211743e-03,  1.37578014e-03,
         -2.65918638e-03, -4.59682825e-03,  1.08266685e-02,
          3.19225286e-04, -3.39815704e-04,  3.00515724e-03,
         -4.54283124e-03,  1.99091917e-03,  3.04673404e-04,
         -2.06009469e-03, -5.83496460e-03, -3.20082905e-04,
         -7.36969912e-03,  2.62772125e-03,  2.83581802e-03,
          3.61718250e-03,  5.98645992e-0

In [26]:
np.matrix(np.cov(closing_price_data[0:21]))

matrix([[2298.87831194, 2262.00182734, 2248.97002129, 2244.25424915,
         2264.72287197, 2242.0785734 , 2247.00507427, 2218.20965277,
         2210.57656197, 2191.60766844, 2169.3601316 , 2208.63183428,
         2240.66668428, 2244.4637656 , 2221.52060189, 2196.70840228,
         2228.25665293, 2222.66670823, 2236.61097744, 2258.73558858,
         2226.8207778 ],
        [2262.00182734, 2226.37501717, 2213.4657342 , 2208.75012508,
         2229.00771267, 2206.74876059, 2211.65848573, 2183.11495595,
         2175.45476787, 2156.75915998, 2134.73764336, 2173.65899375,
         2205.26471881, 2208.87896754, 2186.38073709, 2161.75634514,
         2192.72817646, 2187.20885002, 2200.84920403, 2222.80096555,
         2191.27432219],
        [2248.97002129, 2213.4657342 , 2201.13901578, 2196.56384123,
         2216.81814721, 2194.73337539, 2199.63928556, 2171.25372035,
         2163.69436343, 2145.14606585, 2123.2355284 , 2161.72622788,
         2193.19116917, 2196.84003827, 2174.50049926,

In [24]:
np.matrix(np.ones(21))

matrix([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1.]])

In [40]:
np.linalg.inv(np.matmul(
    np.matmul(np.matrix(np.ones(21)),
              np.matrix(np.cov(closing_price_data[0:21]))),
    np.transpose(np.matrix(np.ones(21)))))

matrix([[1.04410818e-06]])

In [37]:
np.linalg.inv(np.matrix(np.ones(21)))

LinAlgError: Last 2 dimensions of the array must be square

In [45]:
import cvxpy as cvx

s, _ = sigma.shape

w = cvx.Variable(s)# the length of w is x


risk = cvx.quad_form(w, sigma.as_matrix())
prob = cvx.Problem(cvx.Minimize(risk), 
               [cvx.sum_entries(w) == 1])
prob.solve()
w.value

matrix([[-0.01757787],
        [-0.01039326],
        [-0.01285856],
        [ 0.02431759],
        [-0.00747677],
        [ 0.03356881],
        [-0.02832462],
        [-0.02237571],
        [-0.030682  ],
        [-0.00449003],
        [ 0.02720968],
        [-0.06089591],
        [ 0.0057262 ],
        [-0.00292014],
        [ 0.01856009],
        [ 0.01734566],
        [-0.03385171],
        [ 0.00327834],
        [ 0.01535512],
        [-0.00305865],
        [ 0.03497532],
        [ 0.01237246],
        [-0.01630859],
        [ 0.04644744],
        [ 0.00035687],
        [ 0.00340931],
        [-0.0152096 ],
        [-0.00759379],
        [ 0.01443933],
        [-0.00423487],
        [ 0.00308116],
        [-0.00996802],
        [ 0.0346015 ],
        [-0.00166277],
        [-0.05139969],
        [-0.05969947],
        [-0.01643916],
        [-0.01163794],
        [-0.00427971],
        [-0.00297495],
        [ 0.00797524],
        [-0.00820198],
        [-0.01370058],
        [ 0

In [102]:
closing_price_data[0:1]

Name,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2013-05-21,31.814,17.2883,86.3569,57.3325,39.1223,50.7079,33.8299,74.3556,44.02,40.8008,...,28.9535,34.8741,77.7614,41.1599,20.7467,26.7985,45.5818,77.6149,26.3824,31.7795


In [103]:
closing_price_data[21:22]

Name,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2013-06-20,28.7993,16.1133,82.1564,54.3565,34.5157,50.2347,31.5429,71.324,44.895,39.4394,...,27.2069,34.07,74.6191,39.3407,21.2297,24.6602,45.0687,72.2783,26.2772,29.0383


In [110]:
np.log(np.matrix(closing_price_data[21:22])/np.matrix(closing_price_data[0:1]))

matrix([[-9.95553632e-02, -7.03849542e-02, -4.98639610e-02,
         -5.33034515e-02, -1.25278344e-01, -9.37569403e-03,
         -6.99965028e-02, -4.16261128e-02,  1.96823539e-02,
         -3.39363724e-02, -6.24100926e-02, -3.42502652e-02,
         -4.40220055e-02, -8.11105134e-02, -8.93890477e-02,
         -1.33532439e-01, -1.28709235e-01,  9.08420726e-03,
          1.96842312e-02, -7.97486537e-02, -1.54671916e-02,
         -1.86181573e-01, -2.52002851e-03, -4.92804943e-02,
         -1.36436260e-01, -6.88115072e-02, -7.28186891e-02,
         -8.89065744e-02, -4.73606430e-02, -1.48856450e-01,
          2.69028802e-02, -3.80273956e-02, -7.04649439e-02,
         -3.49310973e-02, -7.94348363e-02, -3.07638370e-02,
         -1.80020381e-01,  1.68914213e-02, -1.31210207e-01,
         -3.31924697e-02, -3.00232299e-03, -5.14693521e-02,
         -7.37995206e-02,  6.73351208e-03, -9.37296185e-02,
         -6.13307859e-03, -6.75096941e-02, -1.96754526e-01,
         -8.88825695e-02, -1.14895577e-0

In [108]:
np.matrix(closing_price_data[21:22])

matrix([[ 28.7993,  16.1133,  82.1564,  54.3565,  34.5157,  50.2347,
          31.5429,  71.324 ,  44.895 ,  39.4394,  28.7754,  61.0041,
         170.2642,  34.56  ,  27.3437,  35.8509,   9.7598,  57.58  ,
          25.4723, 117.6043,  41.0459,  23.3658,  45.4184,  36.9534,
          41.25  ,  57.6057,  35.48  ,  24.8595,  42.6288,  88.47  ,
          13.9164,   3.87  ,  39.4346, 160.1565,  85.697 ,  70.1237,
          64.534 , 273.44  ,  49.8276,  72.89  ,  71.6368,  59.3787,
          17.3937,  77.9479,  79.7067,  83.2585,  36.2554,  53.3707,
          17.0765,  13.4761, 109.3617,  34.2101,  37.5256,  35.2818,
          67.7743,  74.6038, 412.65  ,  87.753 ,  12.1915,  33.8604,
          28.6949,  22.9692,  88.8254,  38.8056,  25.076 , 199.58  ,
          26.5115, 224.6368,  20.4695, 111.41  ,   9.11  ,  39.2915,
          84.127 ,  46.1814,  23.4694,  22.4789,  42.2425,  71.1271,
          77.2062,  39.5807,  44.4545,  58.047 ,  29.4201,  14.4   ,
          56.755 ,  47.265 ,  31.8